In [151]:
import pandas as pd
import os

from langchain_qdrant import QdrantVectorStore

from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

In [152]:
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [153]:
from dotenv import load_dotenv
load_dotenv()

True

In [154]:
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings

model_name = "nlpai-lab/KURE-v1"

hf_embeddings = HuggingFaceEndpointEmbeddings(
    model=model_name,
    task="feature-extraction",
    huggingfacehub_api_token=os.environ["HUGGINGFACEHUB_API_TOKEN"],
)

In [117]:
client = QdrantClient(
    url=os.environ["QDRANT_URL"],
    api_key=os.environ["QDRANT_api_key"],
    # port=6333,
    # path = "/Users/hongbikim/Dev/dacon_finance"
)

collections = client.get_collections()
collections

CollectionsResponse(collections=[CollectionDescription(name='rag-finance')])

In [155]:
vector_store = QdrantVectorStore(
    client = client,
    collection_name = "rag-finance",
    embedding = hf_embeddings
)

In [156]:
collection_info = client.get_collection("rag-finance")
print(f"포인터 개수: {collection_info.points_count}")
print(f"컬렉션 상태: {collection_info.status}")

포인터 개수: 11971
컬렉션 상태: green


In [157]:
test = pd.read_csv("/Users/hongbikim/Dev/dacon_finance/open/test.csv")
test

,ID,Question
0,TEST_000,금융산업의 이해와 관련하여 금융투자업의 구분에 해당하지 않는 것은?\n1 소비자금융...
1,TEST_001,위험 관리 계획 수립 시 고려해야 할 요소로 적절하지 않은 것은?\n1 수행인력\n...
2,TEST_002,관리체계 수립 및 운영'의 '정책 수립' 단계에서 가장 중요한 요소는 무엇인가?\n...
3,TEST_003,재해 복구 계획 수립 시 고려해야 할 요소로 옳지 않은 것은?\n1 복구 절차 수립...
4,TEST_004,트로이 목마(Trojan) 기반 원격제어 악성코드(RAT)의 특징과 주요 탐지 지표...
...,...,...
510,TEST_510,"""정보보호최고책임자""의 임명에 관한 설명으로 옳지 않은 것은?\n1 정보보호최고책임..."
511,TEST_511,IPv6 주소 체계의 주요 특징으로 옳지 않은 것은?\n1 NAT 필요성 감소\n2...
512,TEST_512,하이브리드 위협에 대한 설명으로 가장 적절한 것은?\n1 사이버 공간에서만 발생하는...
513,TEST_513,전자금융거래법의 주요 목적 중 하나는 무엇인가?\n1 전자금융거래의 비대면성 강화\...


In [33]:
for qq in test['Question']:
    if '\n' not in qq:
        print(qq)

트로이 목마(Trojan) 기반 원격제어 악성코드(RAT)의 특징과 주요 탐지 지표를 설명하세요.
전자금융거래법에 따라 이용자가 금융 분쟁조정을 신청할 수 있는 기관을 기술하세요.
딥페이크 기술 악용에 대비한 금융권의 대응 방안을 기술하세요.
디지털 지갑(Digital Wallet)에서 우려되는 주요 보안 위협을 설명하세요.
개인정보 접근 권한 검토는 어떻게 수행해야 하며, 그 목적은 무엇인가요?
정보보호의 3대 요소에 해당하는 보안 목표를 3가지 기술하세요.
금융회사가 정보보호 예산을 관리할 때, 전자금융감독규정상 정보기술부문 인력 및 예산의 기준 비율은 얼마인가요?
SMTP 프로토콜의 보안상 주요 역할을 설명하세요.
ID/PW 기반 인증 환경에서 발생할 수 있는 대표적인 공격 유형을 2가지 이상 서술하고 대응 방안을 설명하세요.
전자자금이체의 지급 효력 발생 시점을 전자금융거래법 기준에 따라 설명하세요.
DNSSEC의 주요 기능과 이를 통해 해결할 수 있는 보안 위협을 설명하세요.
VLAN(Virtual LAN)의 주요 기능을 설명하세요.
개인신용평점 하락 가능성과 관련해 설명의무가 발생하는 경우, 금융기관이 설명해야 할 주요 내용은 무엇인가요?
인증 및 권한관리 측면에서 특수 계정에 대한 보안조치 방안은 무엇인가요?
전자금융데이터 보안에서 암호화 방식을 선택할 때 고려해야 할 주요 기준은 무엇인가요?


In [166]:
idx = 0

In [167]:
query = test['Question'][idx].split("\n")[0]

results = vector_store.similarity_search(
    query = query,
    k = 10,
    # filter = models.Filter(must=[models.FieldCondition(
    #     key = "metadata.page", # 특정 필드(예: page) 기반 필터링
    #     match = models.MatchValue(value=2), # page 1인 문서만 검색
    # )])
)

In [168]:
results

[Document(metadata={'file_name': '자본시장과 금융투자업에 관한 법률 시행령(대통령령)(제35391호)(20250919).pdf', 'file_id': '73818cd54012436cb96d18f42a70161f', 'page_chunk_id': 'page_19_chunk_0', 'chunk_length': 339, '_id': '39dbb35d-f02e-4e87-bb8d-6391326ae63f', '_collection_name': 'rag-finance'}, page_content='법제처 20 국가법령정보센터\n자본시장과 금융투자업에 관한 법률 시행령\n1. 별표 1에 따른 금융투자업의 종류 중 다음 각 목의 금융투자업의 경우: 해당 금융투자업에 속하는 금융투자상품\n중 증권, 장내파생상품, 장외파생상품 각각을 기준으로 하여 인가받지 않은 다른 업무 단위\n가. 투자매매업\n나. 투자매매업(인수업은 제외한다)\n다. 투자매매업(인수업만 해당한다)\n2. 별표 1에 따른 금융투자업의 종류 중 투자중개업(같은 표에 따라 인가업무 단위가 2l-1-1, 2l-1-2, 2-14-1 및 2-\n14-2인 투자중개업은 제외한다)의 경우: 투자중개업에 속하는 인가업무 단위 중 인가받지 않은 다른 업무 단위'),
 Document(metadata={'file_name': '자본시장과 금융투자업에 관한 법률(법률)(제20718호)(20250722).pdf', 'file_id': 'd8996ff8506840a3854b8603d3172fd2', 'page_chunk_id': 'page_8_chunk_1', 'chunk_length': 308, '_id': '27858f72-41a3-48bb-885e-0146128e7aa3', '_collection_name': 'rag-finance'}, page_content='1. 금융투자업의 종류(투자매매업, 투자중개업, 집합투자업 및 신탁업을 말하되, 투자매매업 중 인수업을 포함한다)\n2. 금융투자상품(집합투

In [169]:
query = test['Question'][idx]
print(query)
context = []
for result in results:
    context.append(result.page_content)
context = "\n".join(context)
context

금융산업의 이해와 관련하여 금융투자업의 구분에 해당하지 않는 것은?
1 소비자금융업
2 투자자문업
3 투자매매업
4 투자중개업
5 보험중개업


'법제처 20 국가법령정보센터\n자본시장과 금융투자업에 관한 법률 시행령\n1. 별표 1에 따른 금융투자업의 종류 중 다음 각 목의 금융투자업의 경우: 해당 금융투자업에 속하는 금융투자상품\n중 증권, 장내파생상품, 장외파생상품 각각을 기준으로 하여 인가받지 않은 다른 업무 단위\n가. 투자매매업\n나. 투자매매업(인수업은 제외한다)\n다. 투자매매업(인수업만 해당한다)\n2. 별표 1에 따른 금융투자업의 종류 중 투자중개업(같은 표에 따라 인가업무 단위가 2l-1-1, 2l-1-2, 2-14-1 및 2-\n14-2인 투자중개업은 제외한다)의 경우: 투자중개업에 속하는 인가업무 단위 중 인가받지 않은 다른 업무 단위\n1. 금융투자업의 종류(투자매매업, 투자중개업, 집합투자업 및 신탁업을 말하되, 투자매매업 중 인수업을 포함한다)\n2. 금융투자상품(집합투자업의 경우에는 제229조에 따른 집합투자기구의 종류를 말하며, 신탁업의 경우에는 제\n103조제1항 각 호의 신탁재산을 말한다)의 범위(증권, 장내파생상품 및 장외파생상품을 말하되, 증권 중 국채증권\n, 사채권, 그 밖에 대통령령으로 정하는 것을 포함하고 파생상품 중 주권을 기초자산으로 하는 파생상품ㆍ그 밖에\n대통령령으로 정하는 것을 포함한다)\n3. 투자자의 유형(전문투자자 및 일반투자자를 말한다. 이하 같다)\n법제처 9 국가법령정보센터\n자본시장과 금융투자업에 관한 법률 시행령\n무와 관련된 분석정보 등을 제공하는 경우\n10. 다른 법령에 따라 건축물 및 주택의 임대관리 등 부동산의 관리대행, 부동산의 이용ㆍ개발 및 거래에 대한 상담,\n그 밖에 부동산의 투자ㆍ운용에 관한 자문 등의 업무를 영위하는 경우\n\n⑤ 법 제7조제6항에 따라 다음 각 호의 어느 하나에 해당하는 경우에는 해당 호의 금융투자업으로 보지 아니한다.\n\n1. 법 제7조제6항제1호의 경우: 투자중개업\n2. 법 제7조제6항제2호의 경우: 투자매매업\n3. 법 제7조제6항제3호의 경우: 투자매매업 또는 투자중개업\n4. 법 제7

In [170]:
from pydantic import BaseModel, Field
# JSON 스키마 정의
class Answer(BaseModel):
    answer: int = Field(description="정답 번호")
    justification: str = Field(description="정답에 대한 근거")

class Answer_write(BaseModel):
    answer: str = Field(description="정답")
    justification: str = Field(description="정답에 대한 근거")

In [190]:
# Ollama 모델을 불러옵니다.
llm = ChatOllama(model="standard_lee/kanana-nano-2.1b-instruct:latest",
                 temperature=0.01,
                 num_predict = 256)
# llm = ChatOllama(model="qwen3:4b-instruct",
#                  temperature=0.01,
#                  num_predict = 256)

llm2 = ChatOllama(model="standard_lee/kanana-nano-2.1b-instruct:latest",
                 temperature=0.01,
                 num_predict = 1024)

prompt_number = """Your task is to choose correct answer the question based on the context below.
Context: {context}
Question: {query}

If the given context is not relevant to the question, do not refer to it and answer based on your knowledge.

Return your response in the following JSON format:
{{
    "answer": <정답_번호>,
    "evidence": "<정답에_대한_간단한_근거>"
}}

The correct answer number must be returned based on evidence.
Make sure to return only valid JSON without any additional text.
"""

prompt_write = """You are a financial expert.
Your task is to answer the question based on the context below.

Context: {context}
Question: {query}

Write based on the given context.
If the given context is not relevant to the question, do not refer to it and write based on your knowledge.

Be specific and write clearly without repeating.
Return the answer.
"""

In [174]:
query

'트로이 목마(Trojan) 기반 원격제어 악성코드(RAT)의 특징과 주요 탐지 지표를 설명하세요.'

In [139]:
query = "관리체계 수립 및 운영'의 '정책 수립' 단계에서 가장 중요한 요소는 무엇인가?\n1. 정보보호 및 개인정보보호 정책의 제·개정\n2. 경영진의 참여\n3. 최고책임자의 지정\n4. 자원 할당\n5. 내부 감사 절차의 수립"

In [178]:
print(query)
if "\n" in query:
    prompt_template = prompt_number
    json_parser = JsonOutputParser(pydantic_object=Answer)
    prompt = ChatPromptTemplate.from_template(prompt_template)
    chain = prompt | llm | json_parser

else:
    prompt_template = prompt_write
    json_parser = StrOutputParser()
    prompt = ChatPromptTemplate.from_template(prompt_template)
    chain = prompt | llm2 | json_parser


# 간결성을 위해 응답은 터미널에 출력됩니다.
answer = chain.invoke({
        "context": context,
        "query": query
    })
print(answer)

트로이 목마(Trojan) 기반 원격제어 악성코드(RAT)의 특징과 주요 탐지 지표를 설명하세요.
주어진 문맥은 금융투자업 관련 법률 및 규정에 관한 내용으로, 트로이 목마(Trojan) 기반 원격제어 악성코드(RAT)에 대한 특징이나 탐지 지표에 관한 정보를 포함하지 않습니다. 따라서 문맥을 기반으로 답변할 수 없습니다.

트로이 목마(Trojan) 기반 원격제어 악성코드(RAT)의 특징과 주요 탐지 지표는 다음과 같습니다:

- **특징**:  
  - 트로이 목마는 정상적인 프로그램처럼 보이지만, 실제로는 악성 행위를 수행하는 소프트웨어입니다.  
  - RAT는 사용자의 시스템에 접근하여 명령을 받고, 파일을 복사하거나, 로그를 수집하며, 원격으로 시스템을 제어할 수 있습니다.  
  - 일반적으로 사용자에게 무시되거나, 허용된 프로그램처럼 보이기 때문에 감지가 어렵습니다.  
  - 악성코드가 실행되면, 사용자의 개인정보, 보안 정보, 금융 정보 등이 유출될 수 있습니다.

- **주요 탐지 지표**:  
  - **비정상적인 프로세스 실행**: 시스템에서 예상치 못한 프로세스가 실행되는 경우.  
  - **불법 네트워크 연결**: 외부 IP 주소로의 비정상적인 연결 또는 고정된 포트 사용.  
  - **파일 생성 및 수정**: 사용자에게 보이지 않는 디렉터리에 악성 파일이 생성되는 경우.  
  - **명령어 전송 및 반응**: 외부에서 명령을 받고, 시스템에 반응하는 행동.  
  - **메모리 활동 및 네트워크 트래픽 패턴**: 비정상적인 데이터 전송 패턴이나, 대량의 데이터 전송이 발생하는 경우.  
  - **소프트웨어의 기능이 정상과 다름**: 예: 자동화된 작업, 비정상적인 스크린 캡처, 키 로그 수집 등.

이러한 지표를 기반으로 보안 소프트웨어나 방어 시스템이 악성코드를 탐지하고 차단할 수 있습니다.  
(※ 본 답변은 제공된 문맥과 무관하며, 보안 기술에 기반한 일반 지식으로 작성되었습니다.)


In [150]:
context

'② 소요결정기관은 제1항에 따른 소요결정 시 필요하다고 판단하는 경우 소요제기기관, 방첩사, 사이버사, 체계\n운영기관, 사업관리기관 등 관계기관의 의견을 들어 참고할 수 있다. 제2절 획득 단계 제39조(사이버보안조정회의) ① 사업관리기관은 전력지원체계 획득 단계에서 위험관리를 적용하는 경우 다음 각\n호의 사항에 관한 의견을 조율하기 위해 사이버보안조정회의를 구성ㆍ운영할 수 있다.\n1. 보안통제항목의 선정\n2. 보안통제항목의 구현\n3. 보안통제항목 책임기관의 지정\n4. 보안평가 결과에 따른 후속조치 방안의 결정\n5. 그 밖에 보안계획서 작성 및 보안통제항목 선정 등을 위해 의장이 필요하다고 인정하는 사항\n법제처 4 국가법령정보센터\n국방 사이버보안 위험관리 지시\n\n1. 1단계(시스템 보안분류): 시스템 및 해당 시스템에서 생산, 유통하는 정보의 유형에 따라 대상체계를 기밀성,\n무결성, 가용성 수준 또는 기타 특정 유형으로 분류하는 단계를 말한다.\n\n2. 2단계(보안통제항목 선정): 시스템 보안분류 결과를 토대로 보안계획서 작성을 통해 대상체계에 구현해야 하\n는 보안통제항목을 선정하고 구현계획을 수립ㆍ구체화하는 단계를 말한다.\n3. 3단계(보안통제항목 구현): 작성한 보안계획서에 따라 선정된 보안통제항목을 소프트웨어 개발, 장비 구매, 운\n영절차 및 운용환경 마련 등의 방법으로 해당 대상체계에 보안요구사항을 구현하는 단계를 말한다.\n4. 4단계(보안평가): 보안통제항목이 보안계획서에 따라 적절히 구현되었는지 여부를 평가하고 위험판단을 수행\n하는 단계를 말한다.\n5. 5단계(시스템 인가): 보안평가를 통해 도출한 결과에 따라 해당 대상체계의 위험 수준을 결정하고 운용 가능\n여부를 결정하는 단계를 말한다.\n6. 6단계(보안통제항목 모니터링): 운용인가서, 보안계획서, 후속조치계획서 등에 따라 대상체계의 폐기 전까지\n위험을 지속 관리하는 단계를 말한다.\n③ 대상체계에 대한 적용 여부의 판단이 필요한 경우 국방부(지능정보화정책

In [191]:
answer_list = []
for i in range(len(test)):
    query = test['Question'][i]
    print(query)
    context = []
    for result in results:
        context.append(result.page_content)
    context = "\n".join(context)

    if "\n" in query:
        prompt_template = prompt_number
        json_parser = JsonOutputParser(pydantic_object=Answer)
        prompt = ChatPromptTemplate.from_template(prompt_template)
        chain = prompt | llm | json_parser

    else:
        prompt_template = prompt_write
        json_parser = StrOutputParser()
        prompt = ChatPromptTemplate.from_template(prompt_template)
        chain = prompt | llm2 | json_parser


    # 간결성을 위해 응답은 터미널에 출력됩니다.
    answer = chain.invoke({
            "context": context,
            "query": query
        })
    print(answer)
    answer_list.append(answer)

금융산업의 이해와 관련하여 금융투자업의 구분에 해당하지 않는 것은?
1 소비자금융업
2 투자자문업
3 투자매매업
4 투자중개업
5 보험중개업
{'answer': 1, 'evidence': '소비자금융업은 금융투자업의 구분에 해당하지 않습니다. 금융투자업은 주로 투자매매, 투자중개, 집합투자, 투자자문, 투자일임 및 신탁 업무를 포함하며, 소비자금융업은 대출, 예금 등 소비자와의 직접적인 금융 서비스 제공을 목적으로 합니다.'}
위험 관리 계획 수립 시 고려해야 할 요소로 적절하지 않은 것은?
1 수행인력
2 위험 수용
3 위험 대응 전략 선정
4 대상
5 기간
{'answer': 2, 'evidence': '위험 수용은 일반적으로 위험 관리 계획에서 고려해야 할 요소가 아닙니다. 위험 수용은 개인의 투자 성향이나 의사결정에 관련된 개념으로, 위험 관리 계획을 수립할 때는 주로 위험 식별, 평가, 대응 전략 등을 포함합니다.'}
관리체계 수립 및 운영'의 '정책 수립' 단계에서 가장 중요한 요소는 무엇인가?
1 정보보호 및 개인정보보호 정책의 제·개정
2 경영진의 참여
3 최고책임자의 지정
4 자원 할당
5 내부 감사 절차의 수립
{'answer': 2, 'evidence': '경영진의 참여가 정책 수립 단계에서 가장 중요한 요소입니다. 경영진은 조직의 방향성을 설정하고, 전략적 결정을 내리며, 정책의 실행을 관리하는 역할을 하기 때문에 정책 수립 과정에 적극적으로 참여해야 합니다.'}
재해 복구 계획 수립 시 고려해야 할 요소로 옳지 않은 것은?
1 복구 절차 수립
2 비상연락체계 구축
3 개인정보 파기 절차
4 복구 목표시간 정의
{'answer': 3, 'evidence': '개인정보 파기 절차는 재해 복구 계획 수립 시 고려해야 할 요소가 아닙니다. 개인정보 보호는 중요한 사항이지만, 재해 복구 계획에서는 데이터 보존과 복구에 중점을 둡니다.'}
트로이 목마(Trojan) 기반 원격제어 악성코드(RAT)의 특징과 주요 탐지 지표를 설명하세요.
트로이 목마

In [182]:
answer_list

[{'answer': 1,
  'evidence': '소비자금융업은 금융투자업의 구분에 해당하지 않기 때문에 정답은 1번입니다. 소비자금융업은 예금, 대출 등 일반적인 금융 서비스를 제공하는 업무로, 투자매매나 중개와는 다릅니다.'},
 {'answer': 5,
  'evidence': '기간은 위험 관리 계획 수립 시 고려해야 할 요소이지만, 질문에서 명시적으로 언급되지 않았기 때문에 적절하지 않은 선택으로 간주됩니다.'},
 {'answer': 2,
  'evidence': '경영진의 참여가 정책 수립 단계에서 가장 중요한 요소입니다. 경영진은 조직의 방향성을 설정하고, 전략적 결정을 내리며, 정책의 실행을 관리하는 역할을 하기 때문입니다.'},
 {'answer': 3,
  'evidence': '복구 절차 수립, 비상연락체계 구축, 복구 목표시간 정의는 재해 복구 계획 수립 시 고려해야 할 요소이지만, 개인정보 파기 절차는 포함되지 않습니다. 개인정보 파기 절차는 데이터 보존과 관련된 문제로, 재해 복구 계획의 범주에 속하지 않을 수 있습니다.'},
 '트로이 목마(Trojan) 기반 원격제어 악성코드(RAT, Remote Access Tool)의 주요 특징과 탐지 지표는 다음과 같습니다:\n\n### 주요 특징:\n1. **원격 제어 기능**: 사용자 시스템에 침투하여 실시간으로 원격 제어가 가능합니다.\n2. **감염 경로 다양성**: 이메일 첨부 파일, 다운로드된 실행 파일, 웹사이트 링크 등을 통해 감염됩니다.\n3. **은폐 기술**: 시스템 자원을 최소화하거나 사용자에게 눈에 띄지 않게 설치되어 있습니다.\n4. **다양한 변종**: 지속적으로 진화하며 새로운 기능을 추가하거나 기존 기능을 변경합니다.\n\n### 주요 탐지 지표:\n1. **평균 파일 크기 감소**: 악성코드가 시스템 자원을 최소화하기 위해 파일 크기가 줄어듭니다.\n2. **파일 확장자 변화**: 정상 파일의 확장자가 갑자기 변하는 경우(예: .exe에서 .com 등).\

In [59]:
ww = []
for w, qq in enumerate(test['Question']):
    if '\n' not in qq:
        print(w, qq)
        ww.append(w)

4 트로이 목마(Trojan) 기반 원격제어 악성코드(RAT)의 특징과 주요 탐지 지표를 설명하세요.
7 전자금융거래법에 따라 이용자가 금융 분쟁조정을 신청할 수 있는 기관을 기술하세요.
82 딥페이크 기술 악용에 대비한 금융권의 대응 방안을 기술하세요.
101 디지털 지갑(Digital Wallet)에서 우려되는 주요 보안 위협을 설명하세요.
102 개인정보 접근 권한 검토는 어떻게 수행해야 하며, 그 목적은 무엇인가요?
126 정보보호의 3대 요소에 해당하는 보안 목표를 3가지 기술하세요.
131 금융회사가 정보보호 예산을 관리할 때, 전자금융감독규정상 정보기술부문 인력 및 예산의 기준 비율은 얼마인가요?
182 SMTP 프로토콜의 보안상 주요 역할을 설명하세요.
199 ID/PW 기반 인증 환경에서 발생할 수 있는 대표적인 공격 유형을 2가지 이상 서술하고 대응 방안을 설명하세요.
205 전자자금이체의 지급 효력 발생 시점을 전자금융거래법 기준에 따라 설명하세요.
245 DNSSEC의 주요 기능과 이를 통해 해결할 수 있는 보안 위협을 설명하세요.
416 VLAN(Virtual LAN)의 주요 기능을 설명하세요.
458 개인신용평점 하락 가능성과 관련해 설명의무가 발생하는 경우, 금융기관이 설명해야 할 주요 내용은 무엇인가요?
471 인증 및 권한관리 측면에서 특수 계정에 대한 보안조치 방안은 무엇인가요?
507 전자금융데이터 보안에서 암호화 방식을 선택할 때 고려해야 할 주요 기준은 무엇인가요?


In [84]:
for w in e_list:
    query = test['Question'][w]
    print(query)
    context = []
    for result in results:
        context.append(result.page_content)
    context = "\n".join(context)

    if "\n" in query:
        prompt_template = prompt_number
        json_parser = JsonOutputParser(pydantic_object=Answer)
    else:
        prompt_template = prompt_write
        json_parser = StrOutputParser()
        
    prompt = ChatPromptTemplate.from_template(prompt_template)

    # 체인 생성
    chain = prompt | llm | json_parser

    # 간결성을 위해 응답은 터미널에 출력됩니다.
    answer = chain.invoke({
            "context": context,
            "query": query
        })
    print(answer)
    answer_list[w] = answer

키 분배 프로토콜에서 대칭 키 기반 분배 방식의 운영에 대한 설명으로 옳지 않은 것은?
1 대칭 키는 키 분배의 어려움이 있다.
2 대칭 키는 빠른 암호화 속도를 제공한다.
3 대칭 키는 두 개의 키를 사용하여 암호화한다.
4 대칭 키는 동일한 키로 암호화 및 복호화한다.
{'answer': 4, 'justification': '대칭 키는 동일한 키로 암호화 및 복호화한다.'}
서버 접근통제 시스템의 주요 기능으로 옳지 않은 것은?
1 명령어 통제
2 실시간 모니터링
3 서버 하드웨어 장애 자동 복구
4 데이터 전송 속도 최적화
5 사용자별 시스템 접근 권한 통제
{'answer': 5, 'justification': '사용자별 시스템 접근 권한 통제'}
정부가 정보통신망 응용서비스의 개발을 촉진하기 위해 마련해야 할 시책에 포함되지 않는 것은?
1 정보통신망 이용 교육프로그램의 개발 및 보급 지원
2 인터넷 교육에 대한 지원
3 정보통신망 기술인력 양성사업에 대한 지원
4 정보통신망의 안전성 및 신뢰성 제고
5 정보통신망 응용서비스의 개인정보 보호정책 수립
{'answer': 5, 'justification': '정보통신망 응용서비스의 개인정보 보호정책 수립은 이미 정보통신망 이용 교육프로그램의 개발 및 보급 지원, 인터넷 교육에 대한 지원, 정보통신망 기술인력 양성사업에 대한 지원, 정보통신망의 안전성 및 신뢰성 제고가 모두 포함된 범위에서 정부가 마련해야 할 시책에 포함되지 않는다.'}
네트워크 접근통제(NAC)의 주요 기능으로 옳지 않은 것은?
1 데이터 암호화 자동화
2 무선랜과 공유 패스워드 관리
3 네트워크 트래픽 실시간 복호화
4 비인가 단말 반입 차단
5 IP 추적 가능
{'answer': 4, 'justification': 'IP 추적 가능'}
금융 감독기관의 주요 기능으로 옳지 않은 것은?
1 금융기관의 설립 인가
2 금융소비자 보호
3 금융기관의 자산 운용 대행
4 금융상품의 직접 판매
5 금융기관의 검사 및 감독
{'answer': 5,

In [71]:
answer_list

[{'answer': '5',
  'justification': '금융산업의 이해와 관련하여 금융투자업의 구분에 해당하지 않는 것은 보험중개업'},
 {'answer': 4, 'justification': '위험 관리 계획 수립 시 고려해야 할 요소로 적절하지 않은 것은 대상입니다. '},
 {'answer': 4,
  'justification': '자산건전성은 자산 및 부채에 대한 건전성을 별표 7에서 정하는 바에 따라 "정상", "요주의", "고정", "회수의문", "추정손실"의 5단계로 분류하여야 한다.'},
 {'answer': 4,
  'justification': '복구 목표시간 정의는 재해 복구 계획 수립 시 고려해야 할 요소 중 하나입니다. 따라서, 옳지 않은 것은 4번입니다.'},
 '트로이 목마(Trojan) 기반 원격제어 악성코드(RAT)의 특징은 다음과 같습니다.\n\n* 트로이 목마는 일반적으로 사용자에게 숨겨진 위협을 유발하는 악성코드입니다.\n* RAT은 컴퓨터 시스템이나 네트워크를 조종하거나 정보를 훔호하는 데 사용됩니다.\n* 주요 탐지 지표로는 다음과 같습니다.\n\n1. 이상한 패턴의 트래픽\n2. 기존의 네트워크 트래픽 패턴에서 이상한 패턴이 발견되는 경우\n3. 기존의 시스템 상태에서 이상한 상태가 발견되는 경우\n\n이러저 위협을 유발하는 악성코드는 사용자에게 위험한 정보를 숨겨서 위협을 유발하거나 위협을 유발하기 위해 사용자의 컴퓨터나 네트워크를 조종합니다.',
 {'answer': 4, 'justification': '통화신용정책의 수행 및 지급결제제도의 원활한 운영을 위해'},
 {'answer': '2', 'justification': '법정대리인의 동의를 받아야 한다.'},
 '금융지주회사가 금융분쟁조정을 신청할 수 있는 기관을 영위하는 경우에 한하여, 금융분쟁조정위원회(이하 "금융분쟁조정위원"으로 한다)를 말한다.',
 {'answer': '5',
  'justification': '금융권에서 SBOM을 활용

In [195]:
e_list = []
for e, aa in enumerate(answer_list):
    if type(aa) == str:
        continue
    if aa['answer'] == "":
        print(aa)
        e_list.append(e)
e_list

[]

In [185]:
submit = pd.read_csv("/Users/hongbikim/Dev/dacon_finance/open/sample_submission.csv")
submit

,ID,Answer
0,TEST_000,0
1,TEST_001,0
2,TEST_002,0
3,TEST_003,0
4,TEST_004,0
...,...,...
510,TEST_510,0
511,TEST_511,0
512,TEST_512,0
513,TEST_513,0


In [192]:
sub_answer = []
for e, aa in enumerate(answer_list):
    if type(aa) == str:
        sub_answer.append(aa)
        # print(aa)
    else:
        try:
            sub_answer.append(int(aa['answer']))
            # print(int(aa['answer']))
        except Exception as e:
            sub_answer.append(1)
            print(e)

In [193]:
submit['Answer'] = sub_answer
submit

,ID,Answer
0,TEST_000,1
1,TEST_001,2
2,TEST_002,2
3,TEST_003,3
4,TEST_004,트로이 목마(Trojan) 기반 원격제어 악성코드(RAT: Remote Access...
...,...,...
510,TEST_510,1
511,TEST_511,4
512,TEST_512,3
513,TEST_513,4


In [194]:
submit.to_csv("../open/submit_kanana_2b_0827.csv", index=False)

In [189]:
submit_check = pd.read_csv("../open/submit_llama3_0825.csv")
submit_check

,ID,Answer
0,TEST_000,5
1,TEST_001,4
2,TEST_002,4
3,TEST_003,4
4,TEST_004,트로이 목마(Trojan) 기반 원격제어 악성코드(RAT)의 특징은 다음과 같습니다...
...,...,...
510,TEST_510,4
511,TEST_511,4
512,TEST_512,3
513,TEST_513,4


In [103]:
test['Question'][e]

'중앙행정기관의 장이 매년 개인정보 보호를 위한 시행계획을 작성하여 제출해야 하는 기관은 어디인가?\n1 중앙선거관리위원회\n2 보호위원회\n3 국회\n4 헌법재판소'

In [100]:
answer_list[e]

{'answer': '법제처',
 'justification': '중앙행정기관의 장이 매년 개인정보 보호를 위한 시행계획을 작성하여 제출해야 하는 기관은 법제처입니다.'}

In [ ]:
aaa = pd.DataFrame(answer_list)
aaa
aaa.to_pickle("open/answer_list.pkl")

AttributeError: 'str' object has no attribute 'keys'

In [44]:
prompt = ChatPromptTemplate.from_template(prompt_template)

# 체인 생성
chain = prompt | llm | json_parser

# 간결성을 위해 응답은 터미널에 출력됩니다.
answer = chain.invoke({
        "context": context,
        "query": query
    })
print(answer)

{'answer': '5', 'justification': '금융산업의 이해와 관련하여 금융투자업의 구분에 해당하지 않는 것은 보험중개업'}


In [45]:
answer['answer']

'5'

In [ ]:
answer